#Prepared by Tamal Acharya

#Implementation using python and crewai without RAG and chunking

#Define our agents

##Agent Parameters used :

1. Role : Defines the agent’s function within the crew. It determines the kind of tasks the agent is best suited for.

2. Goal : The individual objective that the agent aims to achieve. It guides the agent’s decision-making process.

3. Backstory :Provides context to the agent’s role and goal, enriching the interaction and collaboration dynamics.

4. LLM (optional): Represents the language model that will run the agent. It dynamically fetches the model name from the OPENAI_MODEL_NAME environment variable, defaulting to "gpt-4" if not specified.

5. Verbose (optional): Setting this to True configures the internal logger to provide detailed execution logs, aiding in debugging and monitoring. Default is False.

6. Allow Delegation (optional): Agents can delegate tasks or questions to one another, ensuring that each task is handled by the most suitable agent. Default is True.



In [ ]:
!pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.6 MB/s eta 0:0

In [ ]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.2 MB/s eta 0:00:00


In [ ]:
!pip install langchain-openai

In [ ]:
!pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
from crewai import Agent, Task, Process, Crew
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI # Import ChatGoogleGenerativeAI


GOOGLE_API_KEY = "your_own_api_key"

# Configure environment and LLM - assuming you want to use Google GenAI here too
#os.environ["GOOGLE_API_KEY"] = 'GOOGLE_API_KEY'
#llm = ChatGoogleGenerativeAI(model="gemini/gemini-1.5-flash", temperature=0.3)

from crewai import Agent, LLM

llm = LLM(
    api_key= GOOGLE_API_KEY,
    model="gemini/gemini-1.5-flash",
)

# #Generate and setup the GROQ_API_KEY
# # Define the API key as a variable
# GROQ_API_KEY = "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"
# import os
# # Set the environment variable using the defined variable
# os.environ['GROQ_API_KEY'] = GROQ_API_KEY
# from google.colab import userdata
# import os
# #os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

# #Setup the LLM
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     openai_api_base="https://api.groq.com/openai/v1",
#     openai_api_key=os.environ['GROQ_API_KEY'],
#     model_name="llama3-8b-8192",
#     temperature=0,
#     max_tokens=1000,
# )

# Define your agents with roles and goals
researcher = Agent(
    role='Senior Research Analyst',
    goal='Uncover groundbreaking insights in AI',
    backstory="""You are a Senior Research Analyst at a leading tech think tank.
    Your expertise lies in identifying emerging trends and their potential impact.
    You have a knack for sifting through complex data and presenting actionable findings.""",
    verbose=True,
    allow_delegation=False,
    llm=llm # Explicitly set the LLM for the agent
)
writer = Agent(
    role='Tech Content Strategist',
    goal='Craft compelling content on AI advancements',
    backstory="""You are a renowned Tech Content Strategist, known for your ability to explain complex technical concepts
    in an engaging and understandable manner. You transform raw data and insights into captivating narratives.""",
    verbose=True,
    allow_delegation=False,
    llm=llm # Explicitly set the LLM for the agent
)


#Define Tasks for the agents

##Task Parameters

1. Description: A clear, concise statement of what the task entails.

2. Agent: The agent responsible for the task, assigned either directly or by the crew’s process.

3. Expected Output: A detailed description of what the task’s completion looks like.

4. Tools (optional): The functions or capabilities the agent can utilize to perform the task

5. Context (optional): Specifies tasks whose outputs are used as context for this task.

In [ ]:
# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis of the latest advancements in AI, focusing on
    Transformer models and their applications in natural language processing.
    Identify key trends, breakthrough research, and potential future directions.
    Your final output should be a detailed report on the latest AI breakthroughs.""",
    expected_output="A detailed report summarizing the latest AI breakthroughs, including key trends, breakthrough research, and potential future directions in Transformer models and NLP applications.", # Added expected_output
    agent=researcher
)

task2 = Task(
    description="""Using the insights from the research report, develop a compelling blog post
    that highlights the most significant AI advancements for a tech-savvy audience.
    Your post should be informative, engaging, and easy to understand,
    explaining the impact of these advancements on the future of AI.""",
    expected_output="A compelling and informative blog post, written for a tech-savvy audience, explaining the most significant AI advancements based on the research report.", # Added expected_output
    agent=writer
)

In [ ]:
!pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.4/438.4 kB 24.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
# Instantiate your crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    process=Process.sequential,  # Tasks will be executed in order
    verbose=False
)

# Get your crew to work!
result = crew.kickoff()

print("########################")
result


# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI, focusing on
    Transformer models and their applications in natural language processing.
    Identify key trends, breakthrough research, and potential future directions.
    Your final output should be a detailed report on the latest AI breakthroughs.


# Agent: Senior Research Analyst
## Final Answer: 
**Executive Summary:**

Transformer models have revolutionized Natural Language Processing (NLP), achieving state-of-the-art results in various tasks. This report analyzes recent advancements, focusing on key trends, breakthrough research, and future directions.  We explore improvements in model architecture (e.g., efficient Transformers, Mixture-of-Experts), training methodologies (e.g., reinforcement learning, curriculum learning), and applications (e.g., improved dialogue systems, advanced code generation).  The report concludes by highlighting potential challenges and opport

CrewOutput(raw='## Transformer Models: Reshaping the Landscape of NLP and the Future of AI\n\nThe world of Natural Language Processing (NLP) has been fundamentally reshaped by the advent of Transformer models.  These powerful architectures have consistently shattered performance benchmarks across a wide range of tasks, pushing the boundaries of what\'s possible in AI.  But the revolution isn\'t over; ongoing advancements are paving the way for even more sophisticated and impactful applications.  This post delves into the key trends, breakthrough research, and future directions shaping the exciting trajectory of Transformer models.\n\n\n**The Rise of the Giants (and How to Tame Them):**\n\nOne undeniable trend is the relentless pursuit of scale.  Models boasting hundreds of billions, even trillions, of parameters are demonstrating capabilities previously deemed science fiction.  However, this comes at a significant computational cost.  The good news?  Research is vigorously tackling thi

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided

The error occurs because of how you set your LLM. Using the CrewAI LLM class, which uses LiteLLM in the background, you also need to provide the LLM provider.


For example, if you use the Anthropic Claude 3.5 Sonnet LLM, you need to put anthropic/ before setting the LLM.


from crewai import Agent, LLM

my_llm = LLM(

    api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="anthropic/claude-3-5-sonnet-20240620",
)

my_agent = Agent(

    ...,
    llm=my_llm,
)

ERROR: LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=model='models/gemini-1.5-flash' google_api_key=SecretStr('**********') temperature=0.5 client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000179649639D0> default_metadata=() Pass model as E.g. For 'Huggingface' inference endpoints pass in completion(model='huggingface/starcoder',..) Learn more: https://docs.litellm.ai/docs/providers

If you encounter this error, follow these two steps:

Use the CrewAI LLM class, which leverages LiteLLM in the background.
Make sure to set the LLM provider before configuring the LLM.
For Google AI Studio, use gemini/<LLM name>.

from crewai import Agent, LLM


my_llm = LLM(

    api_key=os.getenv("GEMINI_API_KEY"),
    model="gemini/gemini-1.5-flash",
),

my_agent = Agent(

    ...,
    llm=my_llm,
)


Here are all the major LLM providers, as of today:


1. OpenAI: <LLM name> (Note: You don't have to set the LLM provider.)

2. Azure OpenAI: azure/<your deployment name>

3. Azure AI Studio: azure_ai/<LLM name>

4. Anthropic: anthropic/<LLM name>

5. Mistral API: mistral/<LLM name>

6. Codestral API: codestral/<LLM name>

7. Hugging Face: huggingface/<LLM name>

8. Ollama: ollama/<LLM name>

9. Groq: groq/<LLM name>

10. Vertex AI: vertex_ai/<LLM name>

11. Gemini Google AI Studio: gemini/<LLM name>

12. AWS Sagemaker: sagemaker/<your endpoint name>

13. AWS Bedrock: bedrock/<LLM name>

14. Cohere: cohere/<LLM name>

15. Anyscale: anyscale/<LLM name>

16. Databricks: databricks/<LLM name>

17. Together AI: together_ai/<LLM provider>/<LLM name>

18. Replicate: replicate/<LLM name>

19. Fireworks AI: fireworks_ai/<LLM name>

20. Predibase: predibase/<LLM name>

21. Nvidia NIM: nvidia_nim/<LLM provider>/<LLM name>

22. IBM watsonx.ai: watsonx/<LLM provider>/<LLM name>

23. LiteLLM Proxy (LLM Gateway): litellm_proxy/<LLM name>

In case of an error or if you want to see the full list, please refer to the LiteLLM Providers page. https://docs.litellm.ai/docs/providers